In [ ]:
import numpy as np
import tensorflow as tf
import joblib
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import os
from matplotlib.ticker import MaxNLocator

# Global variables for model and scaler
model = None
scaler = None

In [ ]:
def load_model():
    global model, scaler
    if os.path.exists('solar_lstm_model.h5') and os.path.exists('lstm_scaler.pkl'):
        try:
            model = tf.keras.models.load_model(
                'solar_lstm_model.h5',
                custom_objects={
                    'MeanSquaredError': tf.keras.losses.MeanSquaredError,
                    'MeanAbsoluteError': tf.keras.metrics.MeanAbsoluteError
                },
                compile=False
            )
            scaler = joblib.load('lstm_scaler.pkl')
            return "✅ Model loaded successfully!"
        except Exception as e:
            return f"❌ Error loading model: {str(e)}"
    else:
        return "❌ Model files missing! Please upload model files"

# Initialize model status
status = load_model()

In [ ]:
# Header widget
header = widgets.HTML(value=f"""
    <div style="background-color:#2c3e50; color:white; padding:15px; border-radius:5px; text-align:center;">
        <h1 style="margin:0; font-size:24px;">☀️ SOLAR POWER PREDICTION SYSTEM</h1>
        <p style="margin:5px 0 0; font-size:14px;">{status}</p>
    </div>
    """)

# Sliders for input parameters
sliders = {}
params = [
    ("Ambient Temp (°C)", 0, 50, 25.0, 0.1),
    ("Module Temp (°C)", 0, 50, 35.0, 0.1),
    ("Irradiation (kW/m²)", 0, 1.5, 0.85, 0.01),
    ("Hour (0-23)", 0, 23, 12, 1)
    ]  # Parameter definitions
for label, vmin, vmax, valinit, valstep in params:
    sliders[label] = widgets.FloatSlider(
        value=valinit,
        min=vmin,
        max=vmax,
        step=valstep,
        description=label + ':',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='95%', margin='5px 0')
    )

# Action buttons
buttons = widgets.HBox([
    widgets.Button(
        description='Load Sample',
        button_style='',
        layout=widgets.Layout(width='30%', margin='5px'),
        style={'button_color': '#95a5a6', 'font_weight': 'bold'}
    ),
    widgets.Button(
        description='Reset Inputs',
        button_style='',
        layout=widgets.Layout(width='30%', margin='5px'),
        style={'button_color': '#95a5a6', 'font_weight': 'bold'}
    ),
    widgets.Button(
        description='Predict Power',
        button_style='',
        layout=widgets.Layout(width='30%', margin='5px'),
        style={'button_color': '#3498db', 'font_weight': 'bold'}
    )
])

# Display areas
input_display = widgets.Output(
    layout=widgets.Layout(
    width='100%',
    height='150px',
    border='1px solid #ddd',
    padding='10px',
    margin='10px 0'
)
)
result_display = widgets.HTML(
    value="""
    <div style="background-color:#e3f2fd; padding:15px; border-radius:5px; text-align:center; margin:10px 0;">
        <h3 style="margin:0; color:#0d47a1;">⚡ AC Power Output:</h3>
        <p style="font-size:28px; font-weight:bold; margin:10px 0; color:#0d47a1;">0.00 kW</p>
    </div>
    """
)
viz_output = widgets.Output(layout=widgets.Layout(
    width='100%',
    height='300px',
    border='1px solid #ddd',
    padding='10px',
    margin='10px 0'
))
details_display = widgets.Output(layout=widgets.Layout(
    width='100%',
    height='180px',
    border='1px solid #ddd',
    padding='10px',
    margin='10px 0'
))
footer = widgets.HTML(value="""
    <div style="background-color:#2c3e50; color:white; padding:8px; border-radius:5px; text-align:center; margin-top:15px;">
        <p style="margin:0; font-size:12px;">Solar Power Prediction System • Built with TensorFlow</p>
    </div>
    """)

In [ ]:
# Left panel (input controls)
left_panel = widgets.VBox([widgets.HTML("<h3 style='margin:10px 0; color:#2c3e50;'>WEATHER PARAMETERS</h3>"),
        *sliders.values(),
        buttons,
        widgets.HTML("<h4 style='margin:10px 0; color:#2c3e50;'>CURRENT INPUT VALUES</h4>"),
        input_display
    ],
    layout=widgets.Layout(width='45%', padding='15px', border='1px solid #eee', margin='0 10px 0 0'))

# Right panel (results)
right_panel = widgets.VBox([widgets.HTML("<h3 style='margin:10px 0; color:#2c3e50;'>PREDICTION RESULTS</h3>"),
        result_display,
        widgets.HTML("<h4 style='margin:10px 0; color:#2c3e50;'>PREDICTION VISUALIZATION</h4>"),
        viz_output,
        widgets.HTML("<h4 style='margin:10px 0; color:#2c3e50;'>PREDICTION DETAILS</h4>"),
        details_display
    ],
    layout=widgets.Layout(width='55%', padding='15px', border='1px solid #eee', margin='0 0 0 10px'))

# Main content area
content = widgets.HBox([left_panel, right_panel])

# Full application
app = widgets.VBox([header, content, footer])

In [ ]:
def calculate_temp_diff():
    """Calculate temperature difference between module and ambient"""
    ambient_temp = sliders["Ambient Temp (°C)"].value
    module_temp = sliders["Module Temp (°C)"].value
    return module_temp - ambient_temp

def update_input_display():
    """Update input values table"""
    input_display.clear_output()
    with input_display:
        # Create a table with current values
        table = "<table style='width:100%; border-collapse: collapse;'>"
        table += "<tr style='background-color: #f5f5f5;'><th style='padding:8px; text-align:left; border-bottom:1px solid #ddd;'>Parameter</th><th style='padding:8px; text-align:left; border-bottom:1px solid #ddd;'>Value</th></tr>"

        for label, slider in sliders.items():
            value = slider.value
            if "Temp" in label or "Difference" in label:
                value_str = f"{value:.2f}"
            else:
                value_str = f"{value}"

            table += f"<tr><td style='padding:8px; border-bottom:1px solid #eee;'>{label}</td><td style='padding:8px; border-bottom:1px solid #eee;'>{value_str}</td></tr>"

        # Add calculated temperature difference
        temp_diff = calculate_temp_diff()
        table += f"<tr><td style='padding:8px; border-bottom:1px solid #eee;'>Temp Difference (°C)</td><td style='padding:8px; border-bottom:1px solid #eee;'>{temp_diff:.2f}</td></tr>"
        table += "</table>"
        display(widgets.HTML(table))

def update_visualization(prediction):
    """Update power visualization"""
    viz_output.clear_output()
    with viz_output:
        fig, ax = plt.subplots(figsize=(10, 4))

        # Create a horizontal bar chart
        bars = ax.barh(['Predicted Power'], [prediction], color='#3498db', height=0.6)
        ax.set_xlim(0, max(10, prediction * 1.5))
        ax.set_xlabel('Power Output (kW)', fontsize=12)
        ax.grid(axis='x', linestyle='--', alpha=0.7)

        # Add value labels
        for bar in bars:
            width = bar.get_width()
            ax.text(width + 0.1, bar.get_y() + bar.get_height()/2,
                   f'{width:.2f} kW',
                   ha='left', va='center', fontsize=14, fontweight='bold')

        plt.tight_layout()
        plt.show()

def update_details(prediction):
    """Update prediction details panel"""
    details_display.clear_output()
    with details_display:
        details = "<div style='padding:10px; background-color:#f9f9f9; border-radius:5px;'>"
        details += "<h4 style='margin-top:0; color:#2c3e50;'>Model Input Details:</h4>"
        details += "<ul style='margin-top:0;'>"

        for label, slider in sliders.items():
            value = slider.value
            if "Temp" in label or "Difference" in label:
                value_str = f"{value:.2f}"
            else:
                value_str = f"{value}"

            details += f"<li><b>{label}:</b> {value_str}</li>"

        # Add calculated temperature difference
        temp_diff = calculate_temp_diff()
        details += f"<li><b>Temp Difference (°C):</b> {temp_diff:.2f} (calculated)</li>"

        details += f"</ul><p style='margin:10px 0;'><b>Predicted AC Power:</b> <span style='color:#e74c3c; font-size:18px;'>{prediction:.2f} kW</span></p>"
        details += "</div>"
        display(widgets.HTML(details))

def update_result(prediction):
    """Update main result display"""
    result_display.value = f"""
    <div style="background-color:#e3f2fd; padding:15px; border-radius:5px; text-align:center; margin:10px 0;">
        <h3 style="margin:0; color:#0d47a1;">⚡ AC Power Output:</h3>
        <p style="font-size:28px; font-weight:bold; margin:10px 0; color:#0d47a1;">{prediction:.2f} kW</p>
    </div>
    """

def update_status(message, is_error=False):
    """Update status message in header"""
    color = "#e74c3c" if is_error else "#2ecc71"
    header.value = f"""
    <div style="background-color:#2c3e50; color:white; padding:15px; border-radius:5px; text-align:center;">
        <h1 style="margin:0; font-size:24px;">☀️ SOLAR POWER PREDICTION SYSTEM</h1>
        <p style="margin:5px 0 0; font-size:14px; color:{color};">{message}</p>
    </div>
    """

In [ ]:
def on_load_sample(b):
    """Load sample data into sliders"""
    sliders["Ambient Temp (°C)"].value = 28.5
    sliders["Module Temp (°C)"].value = 38.2
    sliders["Irradiation (kW/m²)"].value = 0.92
    sliders["Hour (0-23)"].value = 14
    update_status("✅ Sample data loaded")
    update_input_display()

def on_reset_inputs(b):
    """Reset sliders to default values"""
    sliders["Ambient Temp (°C)"].value = 25.0
    sliders["Module Temp (°C)"].value = 35.0
    sliders["Irradiation (kW/m²)"].value = 0.85
    sliders["Hour (0-23)"].value = 12
    update_status("✅ Inputs reset to default")
    update_input_display()

def on_predict(b):
    """Make prediction using the model"""
    if model is None or scaler is None:
        update_status("❌ Model not loaded! Cannot make prediction", is_error=True)
        return

    try:
        # Calculate temperature difference
        temp_diff = calculate_temp_diff()

        # Get input values from sliders
        input_df = pd.DataFrame({
            'Ambient Temp (°C)': [sliders["Ambient Temp (°C)"].value],
            'Module Temp (°C)': [sliders["Module Temp (°C)"].value],
            'Irradiation (kW/m²)': [sliders["Irradiation (kW/m²)"].value],
            'Hour (0-23)': [float(sliders["Hour (0-23)"].value)],
            'Temp Difference (°C)': [temp_diff]
        })

        # Extract features and convert to array
        features = input_df.values

        # Repeat the single timestep 6 times to match model input shape
        sequence = np.repeat(features, 6, axis=0)

        # Scale the data
        scaled = scaler.transform(sequence)
        input_sequence = scaled.reshape(1, 6, 5)  # (batch, timesteps, features)

        # Make prediction
        prediction = model.predict(input_sequence).flatten()[0]

        # Update the display
        update_result(prediction)
        update_status("✅ Prediction completed successfully!")
        update_visualization(prediction)
        update_input_display()
        update_details(prediction)

    except Exception as e:
        update_status(f"❌ Prediction failed: {str(e)}", is_error=True)

In [ ]:
# Connect buttons to handlers
buttons.children[0].on_click(on_load_sample)
buttons.children[1].on_click(on_reset_inputs)
buttons.children[2].on_click(on_predict)

# Initial UI setup
update_input_display()

# Initialize visualization placeholder
with viz_output:
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.text(0.5, 0.5, 'Prediction visualization will appear here',
            ha='center', va='center', fontsize=14, color='gray', alpha=0.5)
    ax.axis('off')
    plt.tight_layout()
    plt.show()


# Initialize details placeholder
with details_display:
   display(widgets.HTML(
        "<div style='text-align:center; padding:20px; color:#7f8c8d;'>"
        "Make a prediction to see details"
        "</div>"
    ))

# Display the application
display(app)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
